# Are users home locations GeoVulnerable (Matthew)
1. Brevard (ZoneA)
1. Volusia County
1. Martin, Palm Beach Counties

### Failed?

1. Duvall County

### No Users?

1. Martin, Palm Beach Counties (Inland)

In [95]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np; import geopandas as gpd
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime
from bson import json_util

In [104]:
input_directory  = "/data/chime/geo/matthew/brevard_zone_a_stage_2_temporal_clusters_with_home/"
output_directory = "/data/chime/geo/matthew/brevard_zone_a_geovulnerable"

#### Import the Geometry

In [105]:
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
c = fiona.open('../EvacuationZones/FL_EvacZones/brevard_zone_a.shp','r')
pol = c.next(); zoneA = shape(pol['geometry']).buffer(0); zoneA.is_valid

True

In [106]:
with open('/data/www/chime/tmp/output.geojson', 'w') as oFile:
    json.dump(mapping(zoneA),oFile)

#### Import the user metadata DF (Phasing this out)

In [107]:
_user_meta = pd.read_json(input_directory+'/temporal_clustered_user_meta.json')

In [108]:
_user_meta = _user_meta.sort_index()
_user_meta.head(3)

,home_cluster,tweets,uid,user
0,NaN,4824,563319506,CVSHealthJobs
1,NaN,4117,4754740136,ebbtidebot
2,69.0,3903,2924224280,SONICjobs


## 0. Import all of the individual user dataframes

In [109]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 82 users in /data/chime/geo/matthew/brevard_zone_a_stage_2_temporal_clusters_with_home/


In [110]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    q.put(1)
    return gpd.GeoDataFrame(tweets)

In [111]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 82, 100%

In [112]:
users.sort( key=lambda x: len(x), reverse=True ) 

`users` is an array of user dataframes. Now find which users have _home locations_ in Zone A

In [113]:
x = users[10]
print(len(x))
x.head(1)

179


,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user
0,1.0,"{""type"": ""Point"", ""coordinates"": [-80.19171909...","[-80.1917191, 25.7735786]",2016-08-01 17:56:32+00:00,4,NaN,POINT (-80.1917191 25.7735786),1.0,NaN,I'm jealous of these. But we have a surprises ...,NaN,760172374847287296,494083137,DieEpic


In [114]:
def get_home_cluster_center(userDF):
    return shape(json.loads(userDF.query('cluster=='+str(userDF.home_cluster_id.values[0])).cluster_center.values[0]))

get_home_cluster_center(users[0]).within(zoneA)

False

## 1. Determine who's home cluster center is in Zone A

In [115]:
def insideZoneA(p):
    if p==None:
        return False
    else:
        return p.within(zoneA)

In [116]:
vuln = []
non_vuln = []
for idx, u in enumerate(users):
    if (insideZoneA(get_home_cluster_center(u))):
        vuln.append(u)
    else:
        non_vuln.append(u)
    sys.stderr.write("\r"+str(idx+1))
sys.stderr.write("\rDone...")
sys.stderr.write("Identified {0} vulnerable users and {1} non-vulnerable".format(len(vuln),len(non_vuln)))

Done...Identified 12 vulnerable users and 70 non-vulnerable

In [117]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Write out just the GeoVulnerable, just in case we need them for something later

In [118]:
def safe_mapping(p):
    if p==None or np.isnan(p).any():
        return None
    else:
        return mapping(p)

In [119]:
def safe_json_export(args):
    df, path = args
    df = df.copy()
    uName = df.head(1).user.values[0].lower()
    df['date'] = df['date'].apply(lambda t: datetime.datetime.strftime(t,'%Y-%m-%dT%H:%M:%SZ'))

    clean = df.where((pd.notnull(df)), None)
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in clean.iterrows():
        geom = safe_mapping(row.geometry)
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':row.to_dict()
                    }
        del feature['properties']['geometry']
        geojson['features'].append(feature)
    
    with open(path+"/"+uName+'.geojson','w') as oFile:
        json.dump(geojson, oFile) 

In [120]:
for idx, u in enumerate(vuln):
    safe_json_export( (u, output_directory))
    sys.stderr.write("\r{0} processed".format(idx+1))

12 processed

# 3. Prepare for Analysis

All of these users should already exist in a format prepared for visualizing, these users can be pasted into Google Sheets

In [123]:
for uName in sorted([u.user[0].lower() for u in vuln]):
    print(uName)

babesnunez 7
eproagi 7
haleyraedancer 7
jabhill 7
jimananimal 7
justfiberoptics 7
kevinwoodin 7
lvachna 7
malcolmdenemark 7
mrslick609 7
ongrove 7
sryland1987 7


In [122]:
len(vuln)

12